# Long Short-Term Memory Networks

In this additional challenge, students will build their own LSTM layer from scratch.  
***This one is optional for 1470 students, but can be done for bonus credit!***

---

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf

from preprocess import get_data

In [ ]:
data_path = "../../data"

## Toy Dataset

No spoilers for the preprocessing part of the homework here. 

In [ ]:
example_sentence = "The word <_UNK> is not a common word but flower is a common word"

example_sentence_list = [
    'the', 'word', '<_unk>', 'is', 'not', 
    'a', 'common', 'word', 'but', 'flower', 
    'is', 'a', 'common', 'word']
example_unique_words = [
    '<_unk>', 'a', 'but', 'common', 
    'flower', 'is', 'not', 'the', 'word']
example_w2t_dict = {
    '<_unk>': 0, 'a': 1, 'but': 2, 'common': 3, 'flower': 4, 
    'is': 5, 'not': 6, 'the': 7, 'word': 8}
example_sentence_tokenized = [7, 8, 0, 5, 6, 1, 3, 8, 2, 4, 5, 1, 3, 8]

print(f"1. example_sentence_list \n    {example_sentence_list}\n")
print(f"2. example_unique_words \n    {example_unique_words}\n")
print(f"3. example_w2t_dict \n    {example_w2t_dict}\n")
print(f"4. example_sentence_tokenized \n    {example_sentence_tokenized}\n")

In [ ]:
X_RNN = np.array([
    [7, 8, 0, 5],
    [6, 1, 3, 8],
    [2, 4, 5, 1]])
y_RNN = np.array([
    [8, 0, 5, 6],
    [1, 3, 8, 2],
    [4, 5, 1, 3]])

print(f"X_RNN shape = {X_RNN.shape}")
print(f"y_RNN shape = {y_RNN.shape}")

print(f"X_RNN     --> y_RNN")
for each_X, each_y in zip(X_RNN, y_RNN):
    print(f"{each_X} --> {each_y}")

## Keras LSTM Layer 

We've already looked at `tf.keras.layers.LSTM`'s API. 

- The Keras LSTM Layer expects the input shape to be in the **batch-major form**, `[batch, timesteps, embedding]`. 
- In our language model, `timesteps` is basically our `window`. 
  - That's because we treat a sequence of words as a time-series data.

Also, the most important keywards arguments are `units`, `return_state` and `return_sequences`. 
- `units` is the output embedding size, 
- `return_state` and `return_sequences` are the Boolean variables to return the final state and the sequences of outputs.

In [ ]:
embedding_layer = tf.keras.layers.Embedding(input_dim  = 9, 
                                            output_dim = 2)

X_RNN_embedding = embedding_layer(X_RNN)
batch_size, window_size, embedding_size= X_RNN_embedding.shape ## (3, 4, 2)
print(f"RNN input tokens shape = {X_RNN.shape}")
print(f"RNN embeddings shape   = {X_RNN_embedding.shape}")

We also know that all Keras LSTM layers have the same weight structures, no matter the value of the Boolean flags.

In [ ]:
lstm           = tf.keras.layers.LSTM(units=embedding_size, return_sequences=False, return_state=False)
lstm_state     = tf.keras.layers.LSTM(units=embedding_size, return_sequences=False, return_state=True )
lstm_seq       = tf.keras.layers.LSTM(units=embedding_size, return_sequences=True,  return_state=False)
lstm_seq_state = tf.keras.layers.LSTM(units=embedding_size, return_sequences=True,  return_state=True )

lstm.build(X_RNN_embedding.shape)
lstm_state.build(X_RNN_embedding.shape)
lstm_seq.build(X_RNN_embedding.shape)
lstm_seq_state.build(X_RNN_embedding.shape)

lstm_weights = lstm.get_weights()
lstm_state.set_weights(lstm_weights)
lstm_seq.set_weights(lstm_weights)
lstm_seq_state.set_weights(lstm_weights)

### Keras LSTM Layer Weights

It's time to see how those weights work under the hood. 
- The LSTM weights are in fact three trainable Tensor variables named `kernel`, `recurrent_kernel`. and `bias`.
- `kernel` is the array of weights for the input
- `recurrent_kernel` is the array of weights for the previous hidden state
- `bias` is the array of biases

In [ ]:
for each_weight_tensor in lstm_seq_state.weights:
    print(each_weight_tensor.name)
    print(each_weight_tensor, end = "\n\n")

At this point, you might be wondering 
> but wait a second. Shouldn't there be **four pairs of weights and biases**, <br>
> because there are four internal feed-forward netwroks in a LSTM unit?

And, you are right. There are four pairs of weights and biases for each internal feed-forward network, but the developers of TensorFlow and Keras only decided to put the weights and biases together in a different way. We can reshape them to be make it easier for us. 

In [ ]:
units = embedding_size
W, U, b = lstm_weights

### kernel: weights for the input vector x_{t}
W_i, W_f, W_c, W_o = (
    W[:, :units], W[:, units:(2*units)], W[:, (2*units):(3*units)], W[:, (3*units):])

### recurrent kernel: weights for the previous hidden state h_{t-1}
U_i, U_f, U_c, U_o = (
    U[:, :units], U[:, units:(2*units)], U[:, (2*units):(3*units)], U[:, (3*units):])

### bias
b_i, b_f, b_c, b_o = (
    b[:units], b[units:(units*2)], b[(units*2):(units*3)], b[(units*3):])

## Your Own Implementation of LSTM

Now we can use the weights and biases $W$, $U$, and $b$ in the way that we've covered in the lecture. 

- $x_t$ is the current input at timestep $t$.
- $h_{t-1}$ and $c_{t-1}$ are the previous hidden and cell states. 

\begin{align*}
f_t &= \sigma \left( W_f x_t + U_f h_{t-1} + b_f \right) & \textsf{Forget Module}\\
i_t &= \sigma \left( W_i x_t + U_i h_{t-1} + b_i \right) & \textsf{Remember Module}\\
\tilde{c}_t &= \tanh \left( W_c x_t + U_c h_{t-1} + b_c \right) & \textsf{New Memory}\\
c_t &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t  & \textsf{Cell State Update}\\
o_t &= \sigma \left( W_o x_t + U_o h_{t-1} + b_o \right) & \textsf{Output Module}\\
h_t &= o_t \odot \tanh(c_t)  & \textsf{Output, Hidden State Update}\\
\end{align*}

Now, your job is to finish implementing the `call` method below. 
- The inputs need to be reshaped into the time-major form `[timesteps, batch, embedding]`.
  - This is because is parallelizing the recurrent operations through all timesteps is very difficult.
  - So, we will use the for-loop to advance in the timestep dimension.
  - Then, inside the for-loop, we wil use matrix multiplications for the batch of inputs in the same timestep.
- Remember that, in a single timestep, the input data is in the matrix form with shape `[batch, embedding]`.
  - So, do something like $Y = XW$ instead of $y_i = W x_i, i \in \{1, 2, 3, \cdots\}$.
  - Also, the hidden and cell states are also matrices with the same shape `[batch, embedding]`.
- You should return the whole sequence of outputs and the final hidden and cell states
  - Like when `return_sequences = True` and `return_state = True`.
- The outputs needs to be reshaped back into the batch-major form `[batch, timesteps, embedding]`.

In [ ]:
class MyLSTM(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        super(MyLSTM, self).__init__(**kwargs)
        
    def build(self, input_shape):
        kernel_shape = tf.TensorShape((input_shape[-1], 4*self.units))

        # Create trainable weight variables for this layer.
        self.kernel = self.add_weight(
            name = "kernel", 
            shape = kernel_shape,
            dtype = tf.float32,
            initializer= "glorot_uniform",
            trainable = True)
        self.recurrent_kernel = self.add_weight(
            name = "recurrent_kernel",
            shape = kernel_shape,
            dtype = tf.float32,
            initializer = "orthogonal",
            trainable = True)
        self.bias = self.add_weight(
            name = "bias",
            shape = (4*self.units,),
            dtype = tf.float32,
            initializer = "zeros",
            trainable = True)
        
        # Make sure to call the `build` method at the end
        super().build(input_shape)
        
    def call(self, inputs, initial_state = None):

        ## TODO: Implement LSTM internals

        ## Hidden state and cell state
        if initial_state:
            ht, ct = tf.identity(initial_state[0]), tf.identity(initial_state[1])
        else:
            ht = tf.zeros(shape=(inputs.shape[0], self.units), dtype=tf.float32)
            ct = tf.zeros(shape=(inputs.shape[0], self.units), dtype=tf.float32)
        
        outputs = [] ## we need the whole sequence of outputs
        inputs_time_major = tf.transpose(inputs, perm = [1, 0, 2]) ## swap the batch and timestep axes
        
        return outputs, ht, ct
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.units
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLSTM, self).get_config()
        base_config["units"]   = self.units
        return base_config

## Compare with Keras LSTM Layer

Now we have to see if your LSTM layer returns the exact same outputs as the Keras LSTM layer. 

So, we will initialize your LSTM layer with **the same weights** from the other Keras LSTM layers.

In [ ]:
my_lstm = MyLSTM(units = 2)
my_lstm.build(X_RNN_embedding.shape)
my_lstm.set_weights(lstm_weights)
my_lstm.get_weights()

Then calculate the outputs and the states from your own LSTM layer. 

In [ ]:
# random initial states
tf_generator = tf.random.Generator.from_seed(42)
input_state_h = tf_generator.normal(shape=(1, units))
input_state_c = tf_generator.normal(shape=(1, units))

# initial states in the [batch, embedding] format
ht = tf.repeat(input_state_h, repeats=batch_size, axis=0)
ct = tf.repeat(input_state_c, repeats=batch_size, axis=0)

my_output_seq, my_state_h, my_state_c = my_lstm(X_RNN_embedding, initial_state = (ht, ct))
print(f"my output sequence, shape = {my_output_seq.shape} \n{my_output_seq}\n")
print(f"my final hidden state, shape = {my_state_h.shape} \n{my_state_h}\n")
print(f"my final cell state, shape = {my_state_c.shape} \n{my_state_c}\n")

In [ ]:
keras_output_seq, keras_state_h, keras_state_c = lstm_seq_state(X_RNN_embedding, initial_state = (ht, ct))

print(f"Keras output sequence, shape = {keras_output_seq.shape} \n{keras_output_seq}\n")
print(f"Keras final hidden state, shape = {keras_state_h.shape} \n{keras_state_h}\n")
print(f"Keras final cell state, shape = {keras_state_c.shape} \n{keras_state_c}\n")

If you have implemented your LSTM layer correctly, you will have the same outputs and states within reasonable error margins for the floating point representations.

In [ ]:
print(np.allclose(my_output_seq.numpy(), keras_output_seq.numpy()))
print(np.allclose(my_state_h.numpy(), keras_state_h.numpy()))
print(np.allclose(my_state_c.numpy(), keras_state_c.numpy()))
# use np.isclose for element-wise comparison